In [ ]:
import numpy as np
import sys # Permet de connaitre la version de python
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
from collections import Counter

In [ ]:
#!pip install tqdm

In [ ]:
from tqdm import tqdm # Permet d'afficher les barres de chargement

In [ ]:
with open("bdd.txt","r") as f:
    num_folder = len(f.readlines())
with open("sbt.txt","r") as f:
    if (len(f.readlines()) != num_folder):
        error("[ERROR] number of BDD function does not match BDD number")
print("Number of function/Foler = ", num_folder)  # This would give length of files.

FileNotFoundError: ignored

In [ ]:
print("Get the bdd and sbt list via the bdd.txt & sbt.txt")
with open("bdd.txt") as f:
    tmp_bdd = f.read()
bdd = tmp_bdd.split("\n")

with open("sbt.txt") as f:
    tmp_sbt = f.read()
sbt = tmp_sbt.split("\n")

# Séparation variables / template

Afin de créer une matrice d'encodage, il faut dans un premier temps determiner quelles seront les dimensions de cette matrice. Comme nous encodons en integer encoding les dimmensions des matrices vont etre les suivantes : ( nombre de fichiers, nombre de mot max par fichier ) 

Les counters permettent de compter le nombre d'occurence de chaque mot, ces données sont ensuite utilisé pour determiner si un mot appartient a la template ou si c est une variable.

In [ ]:
bdd_counter =Counter([word for sentence in bdd for word in sentence.split(' ')])
sbt_counter =Counter([word for sentence in sbt for word in sentence.split(' ')])

Une fois le nombre d'occurence de chaque mot calculé, il faut ensuite regarder quels mots sont les plus souvent répété. J'ai considéré que sur 5000 fichiers, si un mot apparait plus de 3500 fois, alors nous pouvons le considerer comme appartenant a la template. 

In [ ]:
'''
common_words_sbt = []
for word in sbt_counter:
    occurence = sbt_counter[word]
    if occurence > 3500:
        common_words_sbt.append(word)
print("common_words_sbt :\n",common_words_sbt)
'''

In [ ]:
'''
common_words_bdd = []
for word in bdd_counter:
    occurence= bdd_counter[word]
    if occurence > 3500:
        common_words_bdd.append(word)
print("common_words_bdd :\n",common_words_bdd)
'''

In [ ]:
common_words_sbt=['', '(', 'MethodDeclaration', 'Modifier', 'public', ')',
                  'static', 'SimpleName', 'Parameter', 'PrimitiveType', 'int', 'boolean', 'BlockStmt',
                  'ExpressionStmt', 'VariableDeclarationExpr', 'VariableDeclarator', 'IntegerLiteralExpr',
                  'res', 'BinaryExpr', 'NameExpr', 'IfStmt', 'ReturnStmt', 'BooleanLiteralExpr', 'true',
                  'false', 'UnaryExpr']
common_words_bdd =  ['Feature:', 'Check', 'the', '', '#', 'result', 'to', 'True', 'Scenario:', 'Given', '=',
                     'and', 'When', 'i', 'call', 'Then', 'should', 'be', 'False']

In [ ]:
'''
def return_common_words(counter):
    common_words = []
    for word in counter:
        print(' --> ',word)
        occurence = counter[word]
        if occurence > 3500:
            common_words.append(word)
    return common_words
    

common_words_bdd = return_common_words(bdd_counter)
common_words_sbt = return_common_words(sbt_counter)
 '''   

# ---------------------------------------------------------------------------------------------------------------

# Listing des variables 

La fonction suivante permet de renvoyer deux tableaux :

Un premier : arr, a deux dimensions, qui permet de lister pour chaque code ses variables, ce code sera encodé puis donné au modèle.
        
le deuxieme : var, permet de lister toutes les variables présent dans tout les codes pour pouvoir ensuite creer un vocabulaire
    

In [ ]:
def return_listing(text,common_words): 
    arr= [[] for _ in range(num_folder)] #pour calculer la taille max de toute les variables réunies 
    var = [] # pour creer un vocabulaire 
    for i,sentence in  enumerate(text):
        text_split = sentence.split( ' ')
        for j,word in enumerate (text_split):
            if word not in common_words:
                arr[i].append(word)
                var.append(word) 
    return arr, var

La fonction suivante permet de retouner pour un unique code SBT la liste des variables et mettre dans le format suivant : 
var1/var2/var3

In [ ]:
def return_variables_unique(text): 
    var = [] # tableau contenant les variables separées par /, ex : var1/var2/...
    text_split = text.split(' ')
    for i,word in enumerate (text_split):
        if word not in common_words_sbt:
            var.append(word)
            var.append('/')
    return ''.join(var)

In [ ]:
variables_unique = return_variables_unique(sbt[0])
#print(variables_unique)

In [ ]:
arr_var_bdd, variables_bdd = return_listing(bdd,common_words_bdd)

In [ ]:
arr_var_sbt,variables_sbt= return_listing(sbt,common_words_sbt)

Conversion des variables en sequence de variables séparées par / ( var1/var2/var3 ... ) 

In [ ]:
def return_var_list(variables_arr):    
    varlist = []

    for vars in variables_arr:
        temp_varlist = []
        for var in vars:
            temp_varlist.append(var)
            temp_varlist.append('/')
        varlist.append(''.join(temp_varlist))
    return varlist

On cree nos séquences de variables avec la fonction précédente

In [ ]:
varlist_bdd = return_var_list(arr_var_bdd)
varlist_sbt = return_var_list(arr_var_sbt)

# Calcul du nombre de var max par code et du nombre de lettre total

In [ ]:
# Pour des raisons qe j'ignore cette cellule creer une erreur losqu'elle est exécutée depuis 'V2_union' avec Jupyter.
# Les résultats obtenus par cette fonciton seront donc entrés en brut sur jupyter pour que le programme puis s'executer. 

def return_nb_var_maxlen_var(arr):
    all_len = []
    nb_var = []
    for words in arr:
        temp_len = 0
        for j,word in enumerate(words):
            for i,char in enumerate(word):
              do_nothing = 0
            temp_len=temp_len+i+1
        nb_var.append(j+1)
        all_len.append(temp_len)
    return max(nb_var), max(all_len)


# Creation d'un vocabulaire pour les variables 

La cellule ci dessous permet de lister chaque caractere different

In [ ]:
vocab_variables_bdd = {l for char in variables_bdd for l in char}
vocab_variables_sbt = {l for char in variables_sbt for l in char}

Nous séparons chacune des variables par un "/" et nous allons ensuite rajouter des start of sentence token et end of sentence token au debut et a la fin de chacune de nos sequences de variables, ces token sont initialement absents du vocabulaire, il faut donc les rajouter manuelement 

In [ ]:
vocab_variables_bdd.add('/')
vocab_variables_bdd.add('sostoken ')
vocab_variables_bdd.add(' eostoken')

vocab_variables_sbt.add('/')

Nous allons maintenant créer des dictionnnaires, c'est à dire associer à chaque caractere un nombre. La cellule suivante creer deux dictionnaire pour chaque type de code ( BDD,SBT) qui nous premettent avec un caractere d'acceder à son index et inversement.

In [ ]:
char_to_ind_bdd = {char : ind for ind,char in enumerate(vocab_variables_bdd)}
ind_to_char_bdd = {ind : char for ind,char in enumerate(vocab_variables_bdd)}

In [ ]:
char_to_ind_sbt = {char : ind for ind,char in enumerate(vocab_variables_sbt)}
ind_to_char_sbt = {ind : char for ind,char in enumerate(vocab_variables_sbt)}

# Valeurs encodage BDD

In [ ]:
#Resultat entrés en brut car la fonction 'return_nb_var_maxlen_var' ne s'execute par depuis 'V2_union'
#number_variables_bdd = 13
#len_chain= 92
number_variables_bdd, len_chain = return_nb_var_maxlen_var(arr_var_bdd)
number_char_variables_bdd = len(vocab_variables_bdd)
len_total_chain_bdd = number_variables_bdd+len_chain+1

'''
print('nombre total de bdd :', num_folder)
print('nombre total de variables par fichier bdd:', number_variables_bdd)
print('taille de la chaine :',len_chain)
print('nombre total de char dans la chaine avec / :',len_total_chain_bdd)
print('nombre de char differents bdd :', number_char_variables_bdd)

'''

"\nprint('nombre total de bdd :', num_folder)\nprint('nombre total de variables par fichier bdd:', number_variables_bdd)\nprint('taille de la chaine :',len_chain)\nprint('nombre total de char dans la chaine avec / :',len_total_chain_bdd)\nprint('nombre de char differents bdd :', number_char_variables_bdd)\n"

# Valeur encodage SBT

In [ ]:
#Resultat entrés en brut car la fonction 'return_nb_var_maxlen_var' ne s'execute par depuis 'V2_union'
#number_variables_sbt=22
#len_chain_sbt=50
number_variables_sbt, len_chain_sbt = return_nb_var_maxlen_var(arr_var_sbt)
number_char_variables_sbt = len(vocab_variables_sbt)
len_total_chain_sbt = number_variables_sbt+len_chain_sbt+1

'''
print('nombre total de bdd :', num_folder)
print('nombre total de variables par fichier sbt:', number_variables_sbt)
print('taille de la chaine :',len_chain_sbt)
print('nombre total de char dans la chaine avec / :',len_total_chain_sbt)
print('nombre de char differents sbt :', number_char_variables_sbt)
'''

"\nprint('nombre total de bdd :', num_folder)\nprint('nombre total de variables par fichier sbt:', number_variables_sbt)\nprint('taille de la chaine :',len_chain_sbt)\nprint('nombre total de char dans la chaine avec / :',len_total_chain_sbt)\nprint('nombre de char differents sbt :', number_char_variables_sbt)\n"

# Description du fonctionnement du modèle.

On convertit nos sequences de variables en dataframe puis nous allosn.
Nous créons en plus deux nouvelles colonne dans notre dataframe, bdd_input et bdd_label. 

Nous allons donc donner au modele pour son entrainement en entrée le SBT et BDD_input, et en sortie le BDD_label que nous allons dans un premier temps convertir en numpy array.

![image.png](attachment:image.png)

le fonctionnement d'un tel model est le suivant : 
Dans un premier temps les informations contenues dans le SBT sont condensée et transformée en un vecteur par l'encoder, ce vecteur est appelé le context vector. 

Le réseau de neuronne récurent qui prend le BDD_input en entrée et le BDD_Label en sortie, nous permet d'indiquer au modele qu'a la suite d'un certain mot, par exemple "je", il doit prédire "suis".

La couche d'attention permet au modele de se focaliser sur certaines partie de la sequence en entrée afin de prédire par exemple le début de la séquence en sortie.

En superposant ces information, le modele est donc capable de savoir que pour un contexte " i am a student" et a la suite de "< s >" il doit prédire " Je " 

< s > représente notre start of sentence token : sostoken

In [ ]:
df = pd.DataFrame(varlist_bdd, columns=['bdd'])
df['sbt'] = varlist_sbt
df['bdd_input'] = varlist_bdd
df['bdd_label'] = varlist_bdd
df.tail(5)

,bdd,sbt,bdd_input,bdd_label
4995,Fonction_4996/bdd_Fonction_4996_1/v/2/m/-1/Fon...,Fonction_4996/Fonction_4996/v/v/m/m/v/v/1/1/f/...,Fonction_4996/bdd_Fonction_4996_1/v/2/m/-1/Fon...,Fonction_4996/bdd_Fonction_4996_1/v/2/m/-1/Fon...
4996,Fonction_4997/bdd_Fonction_4997_1/p/1/h/-8/Fon...,Fonction_4997/Fonction_4997/p/p/h/h/v/v/4/4/h/...,Fonction_4997/bdd_Fonction_4997_1/p/1/h/-8/Fon...,Fonction_4997/bdd_Fonction_4997_1/p/1/h/-8/Fon...
4997,Fonction_4998/bdd_Fonction_4998_1/n/-5/c/-7/Fo...,Fonction_4998/Fonction_4998/n/n/c/c/x/x/1/1/v/...,Fonction_4998/bdd_Fonction_4998_1/n/-5/c/-7/Fo...,Fonction_4998/bdd_Fonction_4998_1/n/-5/c/-7/Fo...
4998,Fonction_4999/bdd_Fonction_4999_1/a/7/a/-11/Fo...,Fonction_4999/Fonction_4999/a/a/a/a/l/l/8/8/y/...,Fonction_4999/bdd_Fonction_4999_1/a/7/a/-11/Fo...,Fonction_4999/bdd_Fonction_4999_1/a/7/a/-11/Fo...
4999,Fonction_5000/bdd_Fonction_5000_1/l/5/k/-23/Fo...,Fonction_5000/Fonction_5000/l/l/k/k/s/s/8/8/b/...,Fonction_5000/bdd_Fonction_5000_1/l/5/k/-23/Fo...,Fonction_5000/bdd_Fonction_5000_1/l/5/k/-23/Fo...


Il faut maintenant convertir nos données en numpy array

In [ ]:

encoder_input = np.array(df.sbt)
decoder_input = np.array(df.bdd_input)
decoder_label = np.array(df.bdd_label)

df.head(10)

,bdd,sbt,bdd_input,bdd_label
0,Fonction_1/bdd_Fonction_1_1/l/-9/b/13/Fonction...,Fonction_1/Fonction_1/l/l/b/b/i/i/4/4/x/x/1/1/...,Fonction_1/bdd_Fonction_1_1/l/-9/b/13/Fonction...,Fonction_1/bdd_Fonction_1_1/l/-9/b/13/Fonction...
1,Fonction_2/bdd_Fonction_2_1/c/0/f/10/Fonction_...,Fonction_2/Fonction_2/c/c/f/f/c/c/6/6/d/d/5/5/...,Fonction_2/bdd_Fonction_2_1/c/0/f/10/Fonction_...,Fonction_2/bdd_Fonction_2_1/c/0/f/10/Fonction_...
2,Fonction_3/bdd_Fonction_3_1/j/3/w/-19/Fonction...,Fonction_3/Fonction_3/j/j/w/w/p/p/9/9/l/l/6/6/...,Fonction_3/bdd_Fonction_3_1/j/3/w/-19/Fonction...,Fonction_3/bdd_Fonction_3_1/j/3/w/-19/Fonction...
3,Fonction_4/bdd_Fonction_4_1/t/1/k/13/Fonction_...,Fonction_4/Fonction_4/t/t/k/k/g/g/8/8/u/u/7/7/...,Fonction_4/bdd_Fonction_4_1/t/1/k/13/Fonction_...,Fonction_4/bdd_Fonction_4_1/t/1/k/13/Fonction_...
4,Fonction_5/bdd_Fonction_5_1/y/5/d/-5/Fonction_...,Fonction_5/Fonction_5/y/y/d/d/l/l/4/4/g/g/3/3/...,Fonction_5/bdd_Fonction_5_1/y/5/d/-5/Fonction_...,Fonction_5/bdd_Fonction_5_1/y/5/d/-5/Fonction_...
5,Fonction_6/bdd_Fonction_6_1/a/-1/j/-3/Fonction...,Fonction_6/Fonction_6/a/a/j/j/e/e/10/10/w/w/7/...,Fonction_6/bdd_Fonction_6_1/a/-1/j/-3/Fonction...,Fonction_6/bdd_Fonction_6_1/a/-1/j/-3/Fonction...
6,Fonction_7/bdd_Fonction_7_1/h/1/x/0/Fonction_7...,Fonction_7/Fonction_7/h/h/x/x/q/q/6/6/g/g/8/8/...,Fonction_7/bdd_Fonction_7_1/h/1/x/0/Fonction_7...,Fonction_7/bdd_Fonction_7_1/h/1/x/0/Fonction_7...
7,Fonction_8/bdd_Fonction_8_1/k/-4/a/0/Fonction_...,Fonction_8/Fonction_8/k/k/a/a/y/y/0/0/e/e/3/3/...,Fonction_8/bdd_Fonction_8_1/k/-4/a/0/Fonction_...,Fonction_8/bdd_Fonction_8_1/k/-4/a/0/Fonction_...
8,Fonction_9/bdd_Fonction_9_1/o/0/f/6/Fonction_9...,Fonction_9/Fonction_9/o/o/f/f/g/g/8/8/c/c/1/1/...,Fonction_9/bdd_Fonction_9_1/o/0/f/6/Fonction_9...,Fonction_9/bdd_Fonction_9_1/o/0/f/6/Fonction_9...
9,Fonction_10/bdd_Fonction_10_1/p/9/r/-1/Fonctio...,Fonction_10/Fonction_10/p/p/r/r/j/j/7/7/i/i/2/...,Fonction_10/bdd_Fonction_10_1/p/9/r/-1/Fonctio...,Fonction_10/bdd_Fonction_10_1/p/9/r/-1/Fonctio...


On repartit nos données en set de test et set d'entrainnement, on choisit de consacrer 90% des données pour l'entrainement et 10% pour le test :

In [ ]:
total = num_folder
test_size = 0.1

train_encoder_input = encoder_input[:-int(total*test_size)]
train_decoder_input = decoder_input[:-int(total*test_size)]
train_decoder_label = decoder_label[:-int(total*test_size)]

test_encoder_input = encoder_input[-int(total*test_size):]
test_decoder_input = decoder_input[-int(total*test_size):]
test_decoder_label = decoder_label[-int(total*test_size):]
'''
print("train dataset shape")
print(train_encoder_input.shape)
print(train_decoder_input.shape)
print(train_decoder_label.shape)

print("\n\ntest dataset shape")
print(test_encoder_input.shape)
print(test_decoder_input.shape)
print(test_decoder_label.shape)
'''

'\nprint("train dataset shape")\nprint(train_encoder_input.shape)\nprint(train_decoder_input.shape)\nprint(train_decoder_label.shape)\n\nprint("\n\ntest dataset shape")\nprint(test_encoder_input.shape)\nprint(test_decoder_input.shape)\nprint(test_decoder_label.shape)\n'

# Encodage des données 

Il faut maintenant encoder nos données, c'est a dire convertir nos suites de variables en une sequence de nombres, cette fonction prend en entrée nos variables et des parametres pour l'encodage et renvoit une version encodée. 

In [ ]:
def encode(variables,char_to_ind, dim1, dim2,type):# type 0 encoder, 1 decoder_input, 2 decoder_label
    encoded_variables_matrix = np.full((dim1,dim2),char_to_ind['/'])
    #print('Encoding Matrix shape : ',encoded_variables_matrix.shape)
    for i,vars in enumerate(variables):
        j=0
        #print(vars)
        if type == 1:
            encoded_variables_matrix[i,j]=char_to_ind['sostoken ']
            j=j+1
        
        for var in (vars):
            #print(var)
            #print(' j :',j)
            encoded_variables_matrix[i,j]=char_to_ind[var]

            j=j+1
        if type == 2:
            encoded_variables_matrix[i,j]=char_to_ind[' eostoken']
            j=j+1

    return encoded_variables_matrix

La fonction suivante permet d'encoder une seule sequence de variables

In [ ]:
def encode_unique_var(text):
    encoded_matrix_unique = np.full(73,char_to_ind_sbt['/'])
    for i,char in enumerate(text):
        encoded_matrix_unique[i]=char_to_ind_sbt[char]

    return encoded_matrix_unique

Nous générons nos données encodées avec les fonction précédentes

In [ ]:
#print("train dataset shape")
train_encoder_input_encoded = encode(train_encoder_input, char_to_ind_sbt,len(train_encoder_input),len_total_chain_sbt,0)
train_decoder_input_encoded = encode(train_decoder_input, char_to_ind_bdd,len(train_decoder_input),len_total_chain_bdd,1)
train_decoder_label_encoded = encode(train_decoder_label, char_to_ind_bdd,len(train_decoder_label),len_total_chain_bdd,2)
#print("\n\ntest dataset shape")

test_encoder_input_encoded = encode(test_encoder_input, char_to_ind_sbt,len(test_encoder_input),len_total_chain_sbt,0)
test_decoder_input_encoded = encode(test_decoder_input, char_to_ind_bdd,len(test_decoder_input),len_total_chain_bdd,1)
test_decoder_label_encoded = encode(test_decoder_label, char_to_ind_bdd,len(test_decoder_label),len_total_chain_bdd,2)

la fonction suivante permet de passer d'une sequence encodée a une sequence lisible de variable

In [ ]:
def decode ( encoded_sequence, ind_to_char):
    decoded_sequence = []
    for var in encoded_sequence:
        decoded_sequence.append(ind_to_char[var])
    return ''.join(decoded_sequence)

In [ ]:
decode(train_decoder_label_encoded[0],ind_to_char_bdd)

# Création du modèle 

In [ ]:
from keras.layers import Input,Embedding,LSTM,Dense,Concatenate,Attention
from keras.models import Model
from keras import backend as K

#hyperparameters
embedding_size = 256
hidden_size = 1024

# trainer model (generator model will use the same encoder tho)
encoder_input = Input(shape=[len_total_chain_sbt]) # size chain SBT 
encoder_embedding = Embedding(len_total_chain_sbt,embedding_size,mask_zero=True)
encoder_embedded = encoder_embedding(encoder_input)

encoder_lstm1 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0)
encoder_output1,encoder_h1,encoder_c1 = encoder_lstm1(encoder_embedded)

encoder_lstm2 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0)
encoder_output2,encoder_h2,encoder_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0)
encoder_output3,encoder_h3,encoder_c3 = encoder_lstm3(encoder_output1)

decoder_input = Input(shape=(None,))
decoder_embedding = Embedding(len_total_chain_bdd,embedding_size,mask_zero=True) # len chain bdd
decoder_embedded = decoder_embedding(decoder_input)

decoder_lstm = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0)
decoder_output,_,_ = decoder_lstm(decoder_embedded,initial_state=[encoder_h3,encoder_c3])

attn_layer = Attention()
attn_context = attn_layer([decoder_output,encoder_output3])

decoder_output = Concatenate(axis=-1)([decoder_output,attn_context])
tanh_dense= Dense(hidden_size*2,activation=K.tanh)
decoder_output = tanh_dense(decoder_output)

softmax_dense = Dense(number_char_variables_bdd,activation='softmax')
decoder_output = softmax_dense(decoder_output)

trainer_model = Model([encoder_input,decoder_input],decoder_output)
trainer_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Entrainement du modèle sur 25 epochs :

In [ ]:
print("\nTraining of the model ... \n")
trainer_hist =trainer_model.fit([train_encoder_input_encoded,train_decoder_input_encoded],train_decoder_label_encoded,epochs=25,batch_size=128,validation_split=0.2, verbose=1)
print("\nTraining completed ! \n")


Sauvegarde des poids du modèle, ce qui permet de pouvoir le charger ( c'est à dire d'etre en mesure de faire des prédictions sans avoir à ré-entrainer le modele ) :

In [ ]:
print("\nSaving the model ... \n")
trainer_model.save_weights('Variables_attention_weights.h5')
print("\nModel saved !\n")
print("\nPLoting the metrics / epochs ... \n")
losses = pd.DataFrame(trainer_model.history.history)
losses.plot()
#trainer_model.load_weights('Variables_attention_weights.h5')


Evalutaion de la précision du modèle : 

In [ ]:
print("\nEvaluating the model ... \n")
score = trainer_model.evaluate([train_encoder_input_encoded,train_decoder_input_encoded], train_decoder_label_encoded, verbose=0)
print("%s: %.2f%%" % (trainer_model.metrics_names[1], score[1]*100))

In [ ]:
#generator model
gen_encoder = Model([encoder_input],[encoder_output3,encoder_h3,encoder_c3])

gen_decoder_values_input = Input(shape=(len_total_chain_sbt,hidden_size))
gen_decoder_h_input = Input(shape=[hidden_size])
gen_decoder_c_input = Input(shape=[hidden_size])

gen_decoder_embedded = decoder_embedding(decoder_input)
gen_decoder_output,gen_decoder_h,gen_decoder_c = decoder_lstm(gen_decoder_embedded,initial_state=[gen_decoder_h_input,gen_decoder_c_input])

attn_context = attn_layer([gen_decoder_output,gen_decoder_values_input])
gen_decoder_output = Concatenate(axis=-1)([gen_decoder_output,attn_context])

gen_decoder_output = tanh_dense(gen_decoder_output)
gen_decoder_output = softmax_dense(gen_decoder_output)

gen_decoder = Model([decoder_input]+[gen_decoder_values_input,gen_decoder_h_input,gen_decoder_c_input],[gen_decoder_output,gen_decoder_h,gen_decoder_c])

# Test

In [ ]:
def generate_from_encoder_input(encoder_input):
    encoder_input = encoder_input.reshape(1,-1)
    values,h,c = gen_encoder.predict(encoder_input)
    
    single_tok = np.zeros((1,1))
    single_tok[0,0] = char_to_ind_bdd['sostoken ']
    decoder_input = single_tok
    
    generated = []
    count = 0
    while(True):
        decoder_output,new_h,new_c = gen_decoder.predict([decoder_input]+[values,h,c])
        count +=1
        
        sampled_index = np.argmax(decoder_output[0,-1,:])
        sampled_word = ind_to_char_bdd[sampled_index]
        
        if sampled_word != ' eostoken' and sampled_index != 0:
            generated.append(sampled_word)
        if count >= len_total_chain_bdd or sampled_word == ' eostoken':
            break
        
        h,c = new_h,new_c
        decoder_input[0,0] = sampled_index
    
    generated = ''.join(generated)
    return generated

In [ ]:
# test de prediciton sur des données 
'''
for i in range(10,20):
    print("\n<<sample encoder input SBT >>")
    print(decode(train_encoder_input_encoded[i],ind_to_char_sbt))
    print("\n")
    print("<<generated BDD >>")
    print(generate_from_encoder_input(train_encoder_input_encoded[i]))
    print("\n")
    print("<<Expected BDD>>")
    print(decode(train_decoder_label_encoded[i],ind_to_char_bdd))
    print("========================================\n")
'''

In [ ]:
def return_generated_var(input_encoded):
    generated = generate_from_encoder_input(input_encoded)
    generated = generated.split('/')
    return generated

In [ ]:
#print(generate_from_encoder_input(train_encoder_input_encoded[0]))


In [ ]:
#return_generated_var(train_encoder_input_encoded[0])